In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv

# Send a GET request to the URL and retrieve the HTML content
url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"
response = requests.get(url)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all movie containers on the page
movie_containers = soup.find_all("div", class_="lister-item-content")

# Prepare the data for CSV file 1
data1 = []
for sno, container in enumerate(movie_containers, start=1):
    title = container.h3.a.text
    director = container.find("p", class_="").find_all("a")
    directors = [d.text for d in director]
    duration = container.find("span", class_="runtime").text.strip(" min")
    year = container.find("span", class_="lister-item-year").text.strip("()")
    rating = container.find("div", class_="ratings-bar").strong.text
    metascore = container.find("span", class_="metascore").text.strip() if container.find("span", class_="metascore") else ''
    data1.append([sno, title, directors, duration, year, rating, metascore])

# Prepare the data for CSV file 2
data2 = []
for container in movie_containers:
    title = container.h3.a.text
    stars = container.find("p", class_="").find_all("a")[1:]
    star_names = [s.text for s in stars]
    votes = container.find("p", class_="sort-num_votes-visible").find_all("span")[1].text.strip("votes")
    genre = container.find("span", class_="genre").text.strip()
    gross = int(container.find('span', attrs={'name': 'nv'})['data-value'][-1]) if container.find('span', attrs={'name': 'nv'}) else 0
    certification = container.find('span', class_='certificate').text if container.find('span', class_='certificate') else ''
    data2.append([title, star_names, votes, genre, gross, certification])

# Write data to CSV file 1
csv_header1 = ["Sno", "Movie Name", "Director Name", "Duration", "Year", "Ratings", "Metascore"]
with open("data1.csv", "w", newline="", encoding="utf-8") as csvfile1:
    writer1 = csv.writer(csvfile1)
    writer1.writerow(csv_header1)
    writer1.writerows(data1)

# Write data to CSV file 2
csv_header2 = ["Movie Name", "Stars", "Votes", "Genre","Gross", "Certification"]
with open("data2.csv", "w", newline="", encoding="utf-8") as csvfile2:
    writer2 = csv.writer(csvfile2)
    writer2.writerow(csv_header2)
    writer2.writerows(data2)


In [2]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("imdb.db")
cursor = conn.cursor()

# Create Table 1 if it doesn't exist
create_table1 = """
CREATE TABLE IF NOT EXISTS Table1 (
    Sno INTEGER PRIMARY KEY,
    MovieName TEXT,
    Director TEXT,
    Duration INTEGER,
    Year TEXT,
    Ratings REAL
);
"""
cursor.execute(create_table1)

# Create Table 2 if it doesn't exist
create_table2 = """
CREATE TABLE IF NOT EXISTS Table2 (
    MovieName TEXT,
    Star TEXT,
    Votes INTEGER,
    Genre TEXT,
    GrossCollection REAL,
    Certification TEXT
);
"""
cursor.execute(create_table2)

# Read data from CSV file 1 and insert into Table 1
with open("data1.csv", mode="r", encoding="utf-8") as file1:
    csv_reader1 = csv.reader(file1)
    next(csv_reader1)  # Skip the header row
    for row in csv_reader1:
        cursor.execute("INSERT INTO Table1 VALUES (?, ?, ?, ?, ?, ?)", row[:6])

# Read data from CSV file 2 and insert into Table 2
with open("data2.csv", mode="r", encoding="utf-8") as file2:
    csv_reader2 = csv.reader(file2)
    next(csv_reader2)  # Skip the header row
    for row in csv_reader2:
        cursor.execute("INSERT INTO Table2 VALUES (?, ?, ?, ?, ?, ?)", row[:6])

# Commit the changes and close the connection
conn.commit()
conn.close()


In [3]:
conn = sqlite3.connect(r"C:\Users\91980\imdb.db")
cursor = conn.cursor()

# Execute the SQL queries and display the results

In [4]:
# 1) Display all the details of movies created by directors Christopher and Matt Reeves.
cursor.execute("SELECT * FROM Table1 WHERE Director LIKE '%Christopher%' OR Director LIKE '%Matt Reeves%'")
results = cursor.fetchall()
for row in results:
    print(row)

(2, 'The Dark Knight', "['Christopher Nolan', 'Christian Bale', 'Heath Ledger', 'Aaron Eckhart', 'Michael Caine']", 152, '2008', 9.0)
(4, 'Inception', "['Christopher Nolan', 'Leonardo DiCaprio', 'Joseph Gordon-Levitt', 'Elliot Page', 'Ken Watanabe']", 148, '2010', 8.8)
(22, 'The Dark Knight Rises', "['Christopher Nolan', 'Christian Bale', 'Tom Hardy', 'Anne Hathaway', 'Gary Oldman']", 164, '2012', 8.4)
(42, 'Batman Begins', "['Christopher Nolan', 'Christian Bale', 'Michael Caine', 'Ken Watanabe', 'Liam Neeson']", 140, '2005', 8.2)


In [5]:
# 2) Display all the details of movies with a duration of 140 minutes to 190 minutes.
cursor.execute("SELECT * FROM Table1 WHERE Duration BETWEEN 140 AND 190")
results = cursor.fetchall()
for row in results:
    print(row)

(1, 'Spider-Man: Across the Spider-Verse', "['Joaquim Dos Santos', 'Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", 140, '2023', 9.1)
(2, 'The Dark Knight', "['Christopher Nolan', 'Christian Bale', 'Heath Ledger', 'Aaron Eckhart', 'Michael Caine']", 152, '2008', 9.0)
(4, 'Inception', "['Christopher Nolan', 'Leonardo DiCaprio', 'Joseph Gordon-Levitt', 'Elliot Page', 'Ken Watanabe']", 148, '2010', 8.8)
(5, 'The Lord of the Rings: The Fellowship of the Ring', "['Peter Jackson', 'Elijah Wood', 'Ian McKellen', 'Orlando Bloom', 'Sean Bean']", 178, '2001', 8.8)
(6, 'The Lord of the Rings: The Two Towers', "['Peter Jackson', 'Elijah Wood', 'Ian McKellen', 'Viggo Mortensen', 'Orlando Bloom']", 179, '2002', 8.8)
(9, 'Soorarai Pottru', "['Sudha Kongara', 'Suriya', 'Paresh Rawal', 'Urvashi', 'Aparna Balamurali']", 153, '2020', 8.7)
(14, 'Sita Ramam', "['Hanu Raghavapudi', 'Dulquer Salmaan', 'Mrunal Thakur', 'Rashmika Mandanna', '

In [6]:
# 3) Display all details of movies with ratings above 7 in ascending order.
cursor.execute("SELECT * FROM Table1 WHERE Ratings > 7 ORDER BY Ratings ASC")
results = cursor.fetchall()
for row in results:
    print(row)

(37, 'Guardians of the Galaxy Vol. 3', "['James Gunn', 'Chris Pratt', 'Chukwudi Iwuji', 'Bradley Cooper', 'Pom Klementieff']", 150, '2023', 8.2)
(38, 'Spider-Man: No Way Home', "['Jon Watts', 'Tom Holland', 'Zendaya', 'Benedict Cumberbatch', 'Jacob Batalon']", 148, '2021', 8.2)
(39, 'Jurassic Park', "['Steven Spielberg', 'Sam Neill', 'Laura Dern', 'Jeff Goldblum', 'Richard Attenborough']", 127, '1993', 8.2)
(40, 'Indiana Jones and the Last Crusade', "['Steven Spielberg', 'Harrison Ford', 'Sean Connery', 'Alison Doody', 'Denholm Elliott']", 127, '1989', 8.2)
(41, '1917', "['Sam Mendes', 'Dean-Charles Chapman', 'George MacKay', 'Daniel Mays', 'Colin Firth']", 119, '2019', 8.2)
(42, 'Batman Begins', "['Christopher Nolan', 'Christian Bale', 'Michael Caine', 'Ken Watanabe', 'Liam Neeson']", 140, '2005', 8.2)
(43, 'Kill Bill: Vol. 1', "['Quentin Tarantino', 'Uma Thurman', 'David Carradine', 'Daryl Hannah', 'Michael Madsen']", 111, '2003', 8.2)
(44, 'Die Hard', "['John McTiernan', 'Bruce Will

In [7]:
# 4) Display all movie names in descending order.
cursor.execute("SELECT MovieName FROM Table1 ORDER BY MovieName DESC")
results = cursor.fetchall()
for row in results:
    print(row)

('Warrior',)
('Vikram Vedha',)
('Vikram',)
('V for Vendetta',)
('Top Gun: Maverick',)
('The Matrix',)
('The Lord of the Rings: The Two Towers',)
('The Lord of the Rings: The Return of the King',)
('The Lord of the Rings: The Fellowship of the Ring',)
('The Dark Knight Rises',)
('The Dark Knight',)
('Terminator 2: Judgment Day',)
('Star Wars: Episode VI - Return of the Jedi',)
('Star Wars: Episode V - The Empire Strikes Back',)
('Star Wars',)
('Spider-Man: No Way Home',)
('Spider-Man: Into the Spider-Verse',)
('Spider-Man: Across the Spider-Verse',)
('Soorarai Pottru',)
('Sita Ramam',)
('Shichinin no samurai',)
('Shershaah',)
('Seppuku',)
('Ratsasan',)
('Ran',)
('Raiders of the Lost Ark',)
('Oldeuboi',)
('North by Northwest',)
('Mononoke-hime',)
('Léon',)
('Kimetsu no Yaiba: Mugen Ressha-Hen',)
('Kill Bill: Vol. 1',)
('Kantara',)
('Kaithi',)
('K.G.F: Chapter 2',)
('Jurassic Park',)
('Indiana Jones and the Last Crusade',)
('Inception',)
('Heat',)
('Guardians of the Galaxy Vol. 3',)
('Gla

In [8]:
# 5) Display movie name starts with ‘P’ and their rating is greater than 7.
cursor.execute("SELECT MovieName FROM Table1 WHERE MovieName LIKE 'P%' AND Ratings > 7")
results = cursor.fetchall()
for row in results:
    print(row)

In [9]:
# 6) Display all movie names with star Arnold Schwarzenegger in ascending order.
cursor.execute("SELECT MovieName FROM Table2 WHERE Star LIKE '%Arnold Schwarzenegger%' ORDER BY MovieName ASC")
results = cursor.fetchall()
for row in results:
    print(row)

('Terminator 2: Judgment Day',)


In [10]:
# 7) Display all details of the movie with the highest number of votes.
cursor.execute("SELECT * FROM Table2 WHERE Votes = (SELECT MAX(Votes) FROM Table2)")
results = cursor.fetchall()
for row in results:
    print(row)

('Gangs of Wasseypur', "['Manoj Bajpayee', 'Nawazuddin Siddiqui', 'Tigmanshu Dhulia', 'Richa Chadha']", '99,425', 'Action, Comedy, Crime', 5.0, 'A')


In [11]:
# 8) Display movie names with gross collections in descending order.
cursor.execute("SELECT MovieName FROM Table2 ORDER BY GrossCollection DESC")
results = cursor.fetchall()
for row in results:
    print(row)

('The Lord of the Rings: The Two Towers',)
('Seppuku',)
('Oldeuboi',)
('Aliens',)
('Top Gun: Maverick',)
('Heat',)
('North by Northwest',)
('Kimetsu no Yaiba: Mugen Ressha-Hen',)
('Inception',)
('Léon',)
('Gladiator',)
('Avengers: Endgame',)
('Avengers: Infinity War',)
('Jurassic Park',)
('Indiana Jones and the Last Crusade',)
('1917',)
('Die Hard',)
('Raiders of the Lost Ark',)
('Shershaah',)
('Warrior',)
('Soorarai Pottru',)
('Spider-Man: Across the Spider-Verse',)
('Star Wars: Episode VI - Return of the Jedi',)
('Kantara',)
('Kill Bill: Vol. 1',)
('V for Vendetta',)
('Gangs of Wasseypur',)
('The Dark Knight',)
('Sita Ramam',)
('Spider-Man: Into the Spider-Verse',)
('The Dark Knight Rises',)
('Kaithi',)
('Asuran',)
('Mononoke-hime',)
('Spider-Man: No Way Home',)
('Ran',)
('Star Wars: Episode V - The Empire Strikes Back',)
('Star Wars',)
('Shichinin no samurai',)
('Ratsasan',)
('Vikram',)
('The Lord of the Rings: The Return of the King',)
('The Lord of the Rings: The Fellowship of the

In [12]:
# 9) Display the gross collection of movies with the star Arnold.
cursor.execute("SELECT GrossCollection FROM Table2 WHERE Star LIKE '%Arnold%'")
results = cursor.fetchall()
for row in results:
    print(row)

(1.0,)


In [13]:
# 10) Display all details of movies with comedy and action genres.
cursor.execute("SELECT * FROM Table2 WHERE Genre LIKE '%Comedy%' AND Genre LIKE '%Action%'")
results = cursor.fetchall()
for row in results:
    print(row)

('Guardians of the Galaxy Vol. 3', "['Chris Pratt', 'Chukwudi Iwuji', 'Bradley Cooper', 'Pom Klementieff']", '161,522', 'Action, Adventure, Comedy', 2.0, 'UA')
('Gangs of Wasseypur', "['Manoj Bajpayee', 'Nawazuddin Siddiqui', 'Tigmanshu Dhulia', 'Richa Chadha']", '99,425', 'Action, Comedy, Crime', 5.0, 'A')


In [14]:
#11) Display all details from both tables where movie names are the same.
cursor.execute("SELECT * FROM table1 JOIN table2 ON table1.MovieName = table2.MovieName")
results=cursor.fetchall()
for row in results:
    print(row)


(1, 'Spider-Man: Across the Spider-Verse', "['Joaquim Dos Santos', 'Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", 140, '2023', 9.1, 'Spider-Man: Across the Spider-Verse', "['Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", '78,165', 'Animation, Action, Adventure', 5.0, 'U')
(2, 'The Dark Knight', "['Christopher Nolan', 'Christian Bale', 'Heath Ledger', 'Aaron Eckhart', 'Michael Caine']", 152, '2008', 9.0, 'The Dark Knight', "['Christian Bale', 'Heath Ledger', 'Aaron Eckhart', 'Michael Caine']", '2,721,664', 'Action, Crime, Drama', 4.0, 'UA')
(3, 'The Lord of the Rings: The Return of the King', "['Peter Jackson', 'Elijah Wood', 'Viggo Mortensen', 'Ian McKellen', 'Orlando Bloom']", 201, '2003', 9.0, 'The Lord of the Rings: The Return of the King', "['Elijah Wood', 'Viggo Mortensen', 'Ian McKellen', 'Orlando Bloom']", '1,888,992', 'Action, Adventure, D

In [15]:
#12)   Display all movie names, Director, ratings, and gross collection where the genre is action.
cursor.execute("SELECT Table1.MovieName, Table1.Director, Table1.Ratings, Table2.GrossCollection FROM Table1 JOIN Table2 ON Table1.MovieName = Table2.MovieName WHERE Table2.Genre LIKE '%Action%'")
results=cursor.fetchall()
for row in results:
    print(row)


('Spider-Man: Across the Spider-Verse', "['Joaquim Dos Santos', 'Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", 9.1, 5.0)
('The Dark Knight', "['Christopher Nolan', 'Christian Bale', 'Heath Ledger', 'Aaron Eckhart', 'Michael Caine']", 9.0, 4.0)
('The Lord of the Rings: The Return of the King', "['Peter Jackson', 'Elijah Wood', 'Viggo Mortensen', 'Ian McKellen', 'Orlando Bloom']", 9.0, 2.0)
('Inception', "['Christopher Nolan', 'Leonardo DiCaprio', 'Joseph Gordon-Levitt', 'Elliot Page', 'Ken Watanabe']", 8.8, 8.0)
('The Lord of the Rings: The Fellowship of the Ring', "['Peter Jackson', 'Elijah Wood', 'Ian McKellen', 'Orlando Bloom', 'Sean Bean']", 8.8, 2.0)
('The Lord of the Rings: The Two Towers', "['Peter Jackson', 'Elijah Wood', 'Ian McKellen', 'Viggo Mortensen', 'Orlando Bloom']", 8.8, 9.0)
('The Matrix', "['Lana Wachowski', 'Lilly Wachowski', 'Keanu Reeves', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Hugo Weaving'

In [16]:
#13)Display all details from both tables with the highest gross collection.
cursor.execute("SELECT * FROM Table1 JOIN Table2 ON Table1.MovieName = Table2.MovieName WHERE Table2.GrossCollection = (SELECT MAX(GrossCollection) FROM Table2)")
results=cursor.fetchall()
for row in results:
    print(row)


(6, 'The Lord of the Rings: The Two Towers', "['Peter Jackson', 'Elijah Wood', 'Ian McKellen', 'Viggo Mortensen', 'Orlando Bloom']", 179, '2002', 8.8, 'The Lord of the Rings: The Two Towers', "['Elijah Wood', 'Ian McKellen', 'Viggo Mortensen', 'Orlando Bloom']", '1,704,929', 'Action, Adventure, Drama', 9.0, 'UA')
(13, 'Seppuku', "['Masaki Kobayashi', 'Tatsuya Nakadai', 'Akira Ishihama', 'Shima Iwashita', 'Tetsurô Tanba']", 133, '1962', 8.6, 'Seppuku', "['Tatsuya Nakadai', 'Akira Ishihama', 'Shima Iwashita', 'Tetsurô Tanba']", '61,119', 'Action, Drama, Mystery', 9.0, '')
(21, 'Oldeuboi', "['Park Chan-wook', 'Choi Min-sik', 'Yoo Ji-tae', 'Kang Hye-jeong', 'Kim Byeong-Ok']", 101, '2003', 8.4, 'Oldeuboi', "['Choi Min-sik', 'Yoo Ji-tae', 'Kang Hye-jeong', 'Kim Byeong-Ok']", '597,089', 'Action, Drama, Mystery', 9.0, 'A')
(23, 'Aliens', "['James Cameron', 'Sigourney Weaver', 'Michael Biehn', 'Carrie Henn', 'Paul Reiser']", 137, '1986', 8.4, 'Aliens', "['Sigourney Weaver', 'Michael Biehn', 'Ca

In [17]:
#14)Display all details from both tables with the highest ratings
cursor.execute("SELECT * FROM Table1 JOIN Table2 ON Table1.MovieName = Table2.MovieName WHERE Table1.Ratings = (SELECT MAX(Ratings) FROM Table1)")
results=cursor.fetchall()
for row in results:
    print(row)

(1, 'Spider-Man: Across the Spider-Verse', "['Joaquim Dos Santos', 'Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", 140, '2023', 9.1, 'Spider-Man: Across the Spider-Verse', "['Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", '78,165', 'Animation, Action, Adventure', 5.0, 'U')


In [18]:
#15)Display all details from both tables with the lowest gross collection and lowest ratings
cursor.execute("SELECT * FROM Table1 JOIN Table2 ON Table1.MovieName = Table2.MovieName WHERE Table1.Ratings = (SELECT MAX(Ratings) FROM Table1)")
results=cursor.fetchall()
for row in results:
    print(row)

(1, 'Spider-Man: Across the Spider-Verse', "['Joaquim Dos Santos', 'Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", 140, '2023', 9.1, 'Spider-Man: Across the Spider-Verse', "['Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez']", '78,165', 'Animation, Action, Adventure', 5.0, 'U')


# using PANDAS SQL 

In [21]:
import pandas as pd
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect(r"C:\Users\91980\imdb.db")

# Function to run a query using Pandas SQL
def run_query(query):
    return pd.read_sql_query(query, conn)

# Query solutions using Pandas SQL

# Query 1
query1 = "SELECT * FROM Table1 WHERE Director IN ('Christopher', 'Matt Reeves')"
result1 = run_query(query1)
print("Query 1 Result:")
print(result1)
print()

# Query 2
query2 = "SELECT * FROM Table1 WHERE Duration BETWEEN 140 AND 190"
result2 = run_query(query2)
print("Query 2 Result:")
print(result2)
print()

# Query 3
query3 = "SELECT * FROM Table1 WHERE Ratings > 7 ORDER BY Ratings ASC"
result3 = run_query(query3)
print("Query 3 Result:")
print(result3)
print()

# Query 4
query4 = "SELECT MovieName FROM Table1 ORDER BY MovieName DESC"
result4 = run_query(query4)
print("Query 4 Result:")
print(result4)
print()

# Query 5
query5 = "SELECT MovieName FROM Table1 WHERE MovieName LIKE 'P%' AND Ratings > 7"
result5 = run_query(query5)
print("Query 5 Result:")
print(result5)
print()

# Query 6
query6 = "SELECT * FROM Table2 WHERE Star LIKE '%Arnold Schwarzenegger%' ORDER BY MovieName ASC"
result6 = run_query(query6)
print("Query 6 Result:")
print(result6)
print()

# Query 7
query7 = "SELECT * FROM Table2 WHERE Votes = (SELECT MAX(Votes) FROM Table2)"
result7 = run_query(query7)
print("Query 7 Result:")
print(result7)
print()

# Query 8
query8 = "SELECT * FROM Table2 ORDER BY GrossCollection DESC"
result8 = run_query(query8)
print("Query 8 Result:")
print(result8)
print()

# Query 9
query9 = "SELECT GrossCollection FROM Table2 WHERE Star LIKE '%Arnold Schwarzenegger%'"
result9 = run_query(query9)
print("Query 9 Result:")
print(result9)
print()

# Query 10
query10 = "SELECT * FROM Table2 WHERE Genre LIKE '%comedy%' AND Genre LIKE '%action%'"
result10 = run_query(query10)
print("Query 10 Result:")
print(result10)
print()



Query 1 Result:
Empty DataFrame
Columns: [Sno, MovieName, Director, Duration, Year, Ratings]
Index: []

Query 2 Result:
    Sno                                          MovieName  \
0     1                Spider-Man: Across the Spider-Verse   
1     2                                    The Dark Knight   
2     4                                          Inception   
3     5  The Lord of the Rings: The Fellowship of the Ring   
4     6              The Lord of the Rings: The Two Towers   
5     9                                    Soorarai Pottru   
6    14                                         Sita Ramam   
7    16                                          Gladiator   
8    19                                  Avengers: Endgame   
9    20                             Avengers: Infinity War   
10   22                              The Dark Knight Rises   
11   24                                             Kaithi   
12   25                                             Asuran   
13   27     

In [27]:
# Query 11 (Subquery)
query11 = "SELECT * FROM Table1 WHERE MovieName IN (SELECT MovieName FROM Table2)"
result11 = run_query(query11)
print("Query 11 Result:")
print(result11)
print()



Query 11 Result:
    Sno                                          MovieName  \
0     1                Spider-Man: Across the Spider-Verse   
1     2                                    The Dark Knight   
2     3      The Lord of the Rings: The Return of the King   
3     4                                          Inception   
4     5  The Lord of the Rings: The Fellowship of the Ring   
5     6              The Lord of the Rings: The Two Towers   
6     7                                         The Matrix   
7     8     Star Wars: Episode V - The Empire Strikes Back   
8     9                                    Soorarai Pottru   
9    10                                          Star Wars   
10   11                         Terminator 2: Judgment Day   
11   12                               Shichinin no samurai   
12   13                                            Seppuku   
13   14                                         Sita Ramam   
14   15                                              

In [28]:
# Query 12 (Subquery)
query12 = "SELECT MovieName, Director, Ratings, GrossCollection FROM Table1 INNER JOIN Table2 ON Table1.MovieName = Table2.MovieName WHERE Table2.Genre LIKE '%action%'"
result12 = run_query(query12)
print("Query 12 Result:")
print(result12)
print()

DatabaseError: Execution failed on sql 'SELECT MovieName, Director, Ratings, GrossCollection FROM Table1 INNER JOIN Table2 ON Table1.MovieName = Table2.MovieName WHERE Table2.Genre LIKE '%action%'': ambiguous column name: MovieName

In [29]:
# Query 13 (Subquery)
query13 = "SELECT * FROM (SELECT * FROM Table1 UNION SELECT * FROM Table2) ORDER BY GrossCollection DESC LIMIT 1"
result13 = run_query(query13)
print("Query 13 Result:")
print(result13)
print()



DatabaseError: Execution failed on sql 'SELECT * FROM (SELECT * FROM Table1 UNION SELECT * FROM Table2) ORDER BY GrossCollection DESC LIMIT 1': no such column: GrossCollection

In [30]:
# Query 14 (Subquery)
query14 = "SELECT * FROM (SELECT * FROM Table1 UNION SELECT * FROM Table2) ORDER BY Ratings DESC LIMIT 1"
result14 = run_query(query14)
print("Query 14 Result:")
print(result14)
print()



Query 14 Result:
      Sno                                          MovieName Director  \
0  Asuran  ['Dhanush', 'Manju Warrier', 'Prakash Raj', 'P...   31,234   

        Duration  Year Ratings  
0  Action, Drama   4.0      UA  



In [31]:
# Query 15 (Subquery)
query15 = "SELECT * FROM (SELECT * FROM Table1 UNION SELECT * FROM Table2) ORDER BY GrossCollection ASC, Ratings ASC LIMIT 1"
result15 = run_query(query15)
print("Query 15 Result:")
print(result15)
print()

DatabaseError: Execution failed on sql 'SELECT * FROM (SELECT * FROM Table1 UNION SELECT * FROM Table2) ORDER BY GrossCollection ASC, Ratings ASC LIMIT 1': no such column: GrossCollection